In [12]:
import csv
import operator
import json
import numpy as np

In [13]:
def readFromCSV(csvPath):
    data = []
    with open(csvPath) as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            data.append(row)
    finalData = []
    for row in data[1:]:
        finalData.append(row[1:])
    return finalData

In [14]:
def sim(data, u: int, v: int):
    chisl = 0
    znam_1 = 0
    znam_2 = 0
    for i in range(len(data[0])):
        if data[u][i] != -1 and data[v][i] != -1:
            chisl += data[u][i] * data[v][i]
            znam_1 += data[u][i] ** 2
            znam_2 += data[v][i] ** 2
    result = chisl / (np.sqrt(znam_1) * np.sqrt(znam_2))
    return result

In [15]:
def mostSimilars(data, user):
        needed = []
        for v in range(0, len(data)):
            if user != v:
                needed.append((v, sim(data, user, v)))
        needed = sorted(needed, key=operator.itemgetter(1), reverse=True);
        return needed

In [16]:
def avg_watched(data, user):
    return sum(i for i in data[user] if i != -1) / sum(1 for i in data[user] if i != -1)

In [17]:
def r_ui(data, user, film, k):
    result = 0
    chisl = 0
    znam = 0

    r_u = avg_watched(data, user)
    
    similar = mostSimilars(data, user)
    
    counter = 0
    for v in similar:
        if counter == k: break
        if data[v[0]][film] != -1:
            r_v = avg_watched(data, v[0])
            chisl += v[1] * ((data[v[0]][film]) - r_v)
            znam += np.abs(v[1])
            counter += 1
    result = chisl / znam + r_u
    return result

In [18]:
def finalize(data, context_place, context_day, k):
    best = {}
    grades = {}
    json = []
    for user_i in range(len(data)):
        task_1 = {}
        for film_i in range(len(data[user_i])):
            if data[user_i][film_i] == -1:
                task_1["movie " + str(film_i + 1)] = round(r_ui(data, user_i, film_i, k), 3)
                if not (user_i + 1 in grades):
                    grades[user_i + 1] = {}
                grades[user_i + 1]["movie " + str(film_i + 1)] = round(r_ui(data, user_i, film_i, k), 3)
        task_2 = {}
        for user_s in mostSimilars(data, user_i):
            if user_i + 1 in best: break
            for film_i in range(len(data[user_i])):
                if data[user_i][film_i] == -1 and context_place[user_s[0]][film_i] == " h" and \
                    (context_day[user_s[0]][film_i] == " Sat" or context_day[user_s[0]][film_i] == " Sun") and \
                    data[user_s[0]][film_i] > avg_watched(data, user_s[0]):
                    best[user_i + 1] = "movie: " + str(film_i + 1)
                    task_2["movie " + str(film_i + 1)] = grades[user_i + 1]['movie ' + str(film_i + 1)]
                    break
        tmp = {'user': user_i + 1, 1: task_1, 2: task_2}
        json.append(tmp)
    return json
    

In [20]:
data = []
for i in readFromCSV("data.csv"):    
    data.append(list(map(int, i)))
context_place = readFromCSV("context_place.csv")
context_day = readFromCSV("context_day.csv")

user_id = int(input("Input user ID: "))
user = finalize(data, context_place, context_day, 4)
final_json = json.dumps(user[user_id - 1])

json_file = open("final.json",'w')
json_file.write(final_json)
json_file.close()
print("JSON created!")

Input user ID: 1
JSON created!
